In [1]:
!pip install beautifulsoup4 
!pip install requests

In [2]:
from bs4 import BeautifulSoup as bs
import mailbox
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import nltk.corpus
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
from google.colab import drive # Для работы с Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def get_df(path):
  dataset = []
  for message in mailbox.mbox(path):

      if message.is_multipart():
          body = []
          for part in message.walk():
              if part.get_content_type() == 'text/plain' or part.get_content_type() == 'text/html':
                  body.append(part.get_payload())
          body = ' '.join(body)
      else:
          body = message.get_payload()
              
      dataset.append({
          'id': message['X-UID'],
          #'agent': message.get_from(),
          'date': message['Date'],
          'from': message['From'],        
          'subject': message['Subject'],
          'content': bs(body, 'html.parser').get_text(),
          'content_type': message['Content-Type'],      
          'raw_message': message
      })
      
  dataset = pd.DataFrame(dataset)
  return dataset

In [7]:
train = get_df('/content/drive/MyDrive/startup/train.mbox')
test = get_df('/content/drive/MyDrive/startup/test.mbox')

train['target'] = 0
test['target'] = 1

df = pd.concat([train,test])
df

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://gasmsgboard.corp.enron.com/msgframe.asp
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.x10.com/home/offer.cgi?!BF1,../bluefish21c.htm?>bf021
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.cob.ohio-state.edu/fin/journal/jofsites.htm
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the d

,id,date,from,subject,content,content_type,raw_message,target
0,22eb6f5f2b4ab14cce43160e3edb0576b62194db,"Tue, 17 Oct 2000 12:12:00 -0700 (PDT)",john.arnold@enron.com,Re: mkts,"It's funny the spreads in gas. First, in gas,...",text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",0
1,cd5283327f06067f3e8f8c38f76942ce3d56d38b,"Sun, 13 May 2001 08:54:00 -0700 (PDT)",john.arnold@enron.com,RE: Extra credit,have you taken any finance courses yet? what's...,text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",0
2,05952d3c79f91678f473181ec07b3faab469fef7,"Wed, 14 Mar 2001 09:17:00 -0800 (PST)",john.arnold@enron.com,Yahoo! Sports Tournament Pick'em,---------------------- Forwarded by John Arnol...,text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",0
3,3d5336e0108e55f071aae9e619d69f468c9d3413,"Mon, 16 Oct 2000 05:40:00 -0700 (PDT)",john.arnold@enron.com,Re: John Arnold's PC,how about 4:00?\n\n\n\n\nIna Rangel\n10/16/200...,text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",0
4,2f1ae7889236050c295354d83fa06b2d9fc7f306,"Wed, 28 Feb 2001 03:15:00 -0800 (PST)",john.arnold@enron.com,Re: mkts,freak show. cash started -7 then to -15 then -...,text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",0
...,...,...,...,...,...,...,...,...
10654,d147daa868084ace58c4921e6911c7fef5712460,"Tue, 2 May 2000 07:36:00 -0700 (PDT)",susan.scott@enron.com,Re: FW: GIR Settlement Hearings - Practice Cro...,Sounds like a great idea -- let us know where/...,text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",1
10655,b59c16dc71d8597df5089d7b137c56b89e5de872,"Mon, 19 Mar 2001 23:59:00 -0800 (PST)",dan.hyvl@enron.com,Re: crestar / gulf aos contract....,"Grant,\n The problem is that they do not want ...",text/plain; charset=us-ascii,"[Date, From, Subject, Content-Type, X-UID]",1
10656,5e718031cc33a08c9c8845f9b3f032d016ef5be6,"Wed, 18 Apr 2001 18:37:00 -0700 (PDT)",james.derrick@enron.com,Question about item #260067307240 AIR-ZONE� PR...,\n\n\n\neBay sent this message to eBay Member....,"multipart/alternative;\n\tboundary=""--26956222...","[Subject, Content-Type, From, Date, X-UID]",1
10657,f98e2fd421519b36376dc6dbb19964e7d9c6cdff,"Fri, 9 Feb 2001 03:34:00 -0800 (PST)",steven.south@enron.com,Verify your Paypal Account,We recently have determined that different com...,"text/html;\n\tcharset=""Windows-1251""","[Subject, Content-Type, From, Date, X-UID]",1


In [16]:
pd.set_option('max_colwidth', 400)

stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

#df['clear_text'] = df['content']
df['clear_text'] = df[['subject','content']].apply(lambda x: str(x['subject']) + ' ' + x['content'], axis=1)
df['clear_text'] = df['clear_text'].str.lower()
df['clear_text'] = df['clear_text'].apply(lambda x: re.sub(r'[\W\d]', " ", x))
#df['clear_text'] = df['clear_text'].apply(lambda x: re.sub(r'[^a-zA-Z]', " ", x))
#df['clear_text'] = df['clear_text'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x)) # Removing Unicode Characters
df['clear_text'] = df['clear_text'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split() if word not in (stop)])) # Лематизация 
df['clear_text'] = df['clear_text'].apply(lambda x: " ".join([word for word in x.split() if len(word) > 2]))
df[['subject','content','clear_text']]

,subject,content,clear_text
0,Re: mkts,"It's funny the spreads in gas. First, in gas, you can play the seasonality \ngame. Does anybody want to buy M/N at 1 back. No, of course not. But in \ncrude, m/n at .10 back is normal. definitely a game as you try to keep month \non month spreads somewhat within reason while preserving year on year spreads \nand creating equilibrium for where cal 2 on back hedging and spec demand is. \nt...",mkts funny spread gas first gas play seasonality game anybody want buy back course crude back normal definitely game try keep month month spread somewhat within reason preserving year year spread creating equilibrium cal back hedging spec demand cal high last time bullish part nat gas curve prompt month remember looking chart always mentality tight july winter think winter spread piece fly int...
1,RE: Extra credit,have you taken any finance courses yet? what's good?\n,extra credit taken finance course yet good
2,Yahoo! Sports Tournament Pick'em,"---------------------- Forwarded by John Arnold/HOU/ECT on 03/14/2001 05:16 \nPM ---------------------------\n\n\n""jfk51272@yahoo.com"" <jfk51272 on 03/14/2001 11:13:41 AM\nTo: jarnold@enron.com\ncc: \nSubject: Yahoo! Sports Tournament Pick'em\n\n\nWednesday, March 14, 09:13:39 AM PT\n\nHello jarnold@enron.com,\n\nYou have been invited to join jfk51272's Private Group in Yahoo!\nSports Tournam...",yahoo sport tournament pick forwarded john arnold hou ect jfk yahoo com jfk jarnold enron com subject yahoo sport tournament pick wednesday march hello jarnold enron com invited join jfk private group yahoo sport tournament pick order join group http tournament fantasysports yahoo com men sign choose join private group prompted enter following information group password done send confirmation ...
3,Re: John Arnold's PC,"how about 4:00?\n\n\n\n\nIna Rangel\n10/16/2000 11:03 AM\nTo: John Arnold/HOU/ECT@ECT\ncc: \nSubject: John Arnold's PC\n\nJohn,\nWhen would be a good time for you. After work maybe?\n\nIna\n---------------------- Forwarded by Ina Rangel/HOU/ECT on 10/16/2000 11:02 AM \n---------------------------\nDON ADAM @ \nENRON \n10/16/2000 10:06 AM\n\nTo: Ina Rangel/HOU/ECT@ECT\ncc: \nSubject: ...",john arnold ina rangel john arnold hou ect ect subject john arnold john would good time work maybe ina forwarded ina rangel hou ect adam enron ina rangel hou ect ect subject john arnold ina could please john email direction house would good time come install equipment thanks
4,Re: mkts,freak show. cash started -7 then to -15 then -7 ended -15.\n,mkts freak show cash started ended
...,...,...,...
10654,Re: FW: GIR Settlement Hearings - Practice Cross Examination,Sounds like a great idea -- let us know where/when.\n,gir settlement hearing practice cross examination sound like great idea let know
10655,Re: crestar / gulf aos contract....,"Grant,\n The problem is that they do not want to assign the agreement. They want to \ndo a partial assignment which is unacceptable unless we can severe the \ndocument so that we have two separate stand alone pieces of paper after the \nassignment. Talk to them about doing two Spot Confirmations, one for 0 to \n8,400 MMBtu per day which would not be assignable, and the other one for 0 to \n1...",crestar gulf aos contract grant problem want assign agreement want partial assignment unacceptable unless severe document two separate stand alone piece paper assignment talk two spot confirmation one mmbtu per day would assignable one mmbtu per day would assignable call discus grant dan hyvl hou ect ect subject crestar gulf aos contract keyspan crestar would like execute existing contract wor...
10656,"Question about item #260067307240 AIR-ZONE� PROFESSIONAL 14,000 mg PRO OZONE GENERATOR!",\n\n\n\neBay sent this message to eBay Member.Your registered nam=\ne is included to show this message originated from eBay. Learn more. \n\n\n\n\n\nQuestion about Item -- Respond Now \n<=\n/TD>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\neBa=\ny sent this message on beh

In [17]:
df.to_pickle('startup2.pkl')

# Предсказание

In [18]:
import pickle
import pandas as pd
import re

import nltk.corpus
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
with open('model1.pkl', 'rb') as f:
    model = pickle.load(f)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your ow

In [22]:
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [26]:
content = 'We recently have determined that different computers have logged \ninto your PayPal account, and multiple password failures were present before \nthe login. One of our Customer Service employees has already tryed to \ntelephonically reach you. As our employee did not manage to reach you, this \nemail has been sent to your notice.Therefore your account has been temporary \nsuspended. '
content = pd.Series(content)
content = content.str.lower()
content = content.apply(lambda x: re.sub(r'[\W\d]', " ", x))
content = content.apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split() if word not in (stop)]))
content = content.apply(lambda x: " ".join([word for word in x.split() if len(word) > 2]))
content

0    recently determined different computer logged paypal account multiple password failure present login one customer service employee already tryed telephonically reach employee manage reach email sent notice therefore account temporary suspended
dtype: object

In [39]:
pred = model.predict_proba(content)[0,1]
pred = int(pred >= 0.85)
pred

1